In [17]:
import pandas as pd
import numpy as np

import os

sleep_sheet = 'Sleep_Efficiency.csv'
input_dir = r'C:\Users\tiakd\.spyder-py3\Homework 4'
sleep_file = os.path.join(input_dir, sleep_sheet)

try:   
    with open(sleep_file) as f:
        print('Q1')
        slp_dataframe = pd.read_csv(f)
    
except Exception as e:
    print(e)
    print('failed to read sleep data: ', sleep_sheet)

Q1


In [22]:
print('Q2')
list_of_cols = [x for x in slp_dataframe]
list_miss_col = []

for col in list_of_cols:  
    #Get the total # of entries
    tot_entries = len(slp_dataframe[col].unique())        
    # Get the total # missing
    count_mis = slp_dataframe[col].isnull().sum()        
        
    if count_mis > 0:
        list_miss_col.append(col)
    
    print(col,'has',tot_entries,'unique entries total.')
        
    print(col,'has',count_mis,'entries missing')
    print()

Q2
ID has 452 unique entries total.
ID has 0 entries missing

Age has 61 unique entries total.
Age has 0 entries missing

Gender has 2 unique entries total.
Gender has 0 entries missing

Bedtime has 424 unique entries total.
Bedtime has 0 entries missing

Wakeup time has 434 unique entries total.
Wakeup time has 0 entries missing

Sleep duration has 9 unique entries total.
Sleep duration has 0 entries missing

Sleep efficiency has 50 unique entries total.
Sleep efficiency has 0 entries missing

REM sleep percentage has 11 unique entries total.
REM sleep percentage has 0 entries missing

Deep sleep percentage has 18 unique entries total.
Deep sleep percentage has 0 entries missing

Light sleep percentage has 21 unique entries total.
Light sleep percentage has 0 entries missing

Awakenings has 6 unique entries total.
Awakenings has 20 entries missing

Caffeine consumption has 7 unique entries total.
Caffeine consumption has 25 entries missing

Alcohol consumption has 7 unique entries tot

In [26]:
print('Q4')
same_gen = slp_dataframe.groupby(['Gender','Age']) 
for x in list_miss_col:
    series_null = np.where(slp_dataframe[x].isnull()==True)[0]        
    
    for y in series_null:
        #Get the row where the data is null
        row = slp_dataframe.iloc[y] 
        #Get the Age of Gender of that person
        age = row['Age']
        gen = row['Gender']

        vals_to_avg = []
        count = 0
        minus_checked = False
        while True:
            
            if (gen,age+count) in same_gen.groups.keys():
                temp = same_gen.get_group((gen,age+count))
            elif (gen,age-count) in same_gen.groups.keys():
                temp = same_gen.get_group((gen,age-count))
                minus_checked = True
            else:
                #The queried age does not exist for the person's gender.  
                count+= 1
                continue
        
            non_nul = np.where(temp[x].isnull()==False)[0]
                
            if len(vals_to_avg) == 5:
                break
                
            for z in non_nul:
                #Z is the location of the non null value in temp
                vals_to_avg.append(temp.iloc[z][x])
                    
                if len(vals_to_avg) == 5:
                    break
            if minus_checked and count > 0:
                count+= 1
                minus_checked = False
            else:
                count+=1
        #Broken out of while loop.  Assign the missing values 
        slp_dataframe.at[y,x] = np.average(vals_to_avg)

Q4


In [27]:
int_ind = pd.IntervalIndex.from_tuples([(0,12),(13,17),(18,30),(31,60),(61,200)])

slp_dataframe['Smoking status'] = np.where(slp_dataframe['Smoking status']=='Yes',1,0)
   
grouper_age = slp_dataframe.groupby(pd.cut(slp_dataframe['Age'], int_ind))
grouper_gen_age = slp_dataframe.groupby([slp_dataframe['Gender'],pd.cut(slp_dataframe['Age'], int_ind)])
    
col = ['Group 1','Group 2','Group 3','Group 4','Group 5']
list_of_cols_eval = ['Age','Sleep duration','Sleep efficiency','REM sleep percentage','Deep sleep percentage',\
                    'Light sleep percentage','Awakenings','Caffeine consumption','Alcohol consumption', \
                    'Smoking status','Exercise frequency']
    
comb_df = {}
for x in list_of_cols_eval:    
    tmp_list = []
    
    for y in range(0,len(int_ind)):    
        val_to_save = (np.round(grouper_age.get_group(int_ind[y])[x].mean(),2),np.round(grouper_age.get_group(int_ind[y])[x].std(),2))
            
        tmp_list.append(val_to_save)
        #Row - Col
        comb_df[x] = tmp_list
    
total_df = pd.DataFrame(comb_df,index=col)
comb = total_df.transpose()
comb.index.names = ['Metrics']

print('Combined')
print(comb)

Combined
                             Group 1       Group 2         Group 3  \
Metrics                                                              
Age                     (10.5, 1.29)  (15.5, 1.29)    (25.8, 2.95)   
Sleep duration          (8.38, 0.95)  (7.62, 0.75)     (7.5, 0.78)   
Sleep efficiency        (0.54, 0.02)  (0.64, 0.04)    (0.77, 0.14)   
REM sleep percentage     (18.0, 0.0)   (19.0, 2.0)   (23.75, 4.19)   
Deep sleep percentage    (35.0, 0.0)   (32.0, 6.0)  (52.81, 16.59)   
Light sleep percentage   (45.0, 0.0)  (47.25, 4.5)  (25.57, 15.53)   
Awakenings               (2.5, 1.29)  (2.25, 0.96)    (1.66, 1.39)   
Caffeine consumption      (0.0, 0.0)  (12.5, 25.0)  (31.86, 34.78)   
Alcohol consumption     (2.75, 1.71)  (2.85, 2.18)    (1.42, 1.82)   
Smoking status            (1.0, 0.0)    (0.0, 0.0)    (0.39, 0.49)   
Exercise frequency        (0.0, 0.0)    (0.0, 0.0)    (1.82, 1.24)   

                               Group 4         Group 5  
Metrics                

In [28]:
women_df = {"Group 1":[],'Group 2':[],'Group 3':[],'Group 4':[],'Group 5':[]}
men_df = {"Group 1":[],'Group 2':[],'Group 3':[],'Group 4':[],'Group 5':[]}
for x in list_of_cols_eval:
    women_list = []
    men_list = []
    for name,group in grouper_gen_age:
        val_to_save = (np.round(grouper_gen_age.get_group(name)[x].mean(),2),np.round(grouper_gen_age.get_group(name)[x].std(),2))
        inter = name[1]
        if name[0] == 'Female':
            if 12 in inter:
                women_df['Group 1'].append(val_to_save)
            elif 17 in inter:
                women_df['Group 2'].append(val_to_save)
            elif 30 in inter:
                women_df['Group 3'].append(val_to_save)
            elif 60 in inter:
                women_df['Group 4'].append(val_to_save)
            else:
                women_df['Group 5'].append(val_to_save)
        else:
            if 12 in inter:
                men_df['Group 1'].append(val_to_save)
            elif 17 in inter:
                men_df['Group 2'].append(val_to_save)
            elif 30 in inter:
                men_df['Group 3'].append(val_to_save)
            elif 60 in inter:
                men_df['Group 4'].append(val_to_save)
            else:
                men_df['Group 5'].append(val_to_save)
                    
women_df = pd.DataFrame(women_df,index=list_of_cols_eval)
women_df.index.names = ['Metrics']

print('Women')
print(women_df)

Women
                             Group 1       Group 2         Group 3  \
Metrics                                                              
Age                     (10.5, 1.29)  (15.5, 1.29)   (26.64, 2.95)   
Sleep duration          (8.38, 0.95)  (7.62, 0.75)    (7.55, 0.87)   
Sleep efficiency        (0.54, 0.02)  (0.64, 0.04)    (0.79, 0.14)   
REM sleep percentage     (18.0, 0.0)   (19.0, 2.0)   (25.11, 3.25)   
Deep sleep percentage    (35.0, 0.0)   (32.0, 6.0)  (53.98, 14.47)   
Light sleep percentage   (45.0, 0.0)  (47.25, 4.5)  (23.83, 14.22)   
Awakenings               (2.5, 1.29)  (2.25, 0.96)    (1.46, 1.35)   
Caffeine consumption      (0.0, 0.0)  (12.5, 25.0)   (37.5, 24.18)   
Alcohol consumption     (2.75, 1.71)  (2.85, 2.18)    (0.93, 1.59)   
Smoking status            (1.0, 0.0)    (0.0, 0.0)    (0.39, 0.49)   
Exercise frequency        (0.0, 0.0)    (0.0, 0.0)    (1.52, 1.33)   

                               Group 4         Group 5  
Metrics                   

In [29]:
men_df['Group 1'] = [(0,0) for x in range(0,11)]
men_df['Group 2'] = [(0,0) for x in range(0,11)]
men_df = pd.DataFrame(men_df,index=list_of_cols_eval)
men_df.index.names = ['Metrics']
print('Men')
print(men_df)

Men
                       Group 1 Group 2         Group 3         Group 4  \
Metrics                                                                  
Age                     (0, 0)  (0, 0)   (24.82, 2.65)   (48.76, 6.82)   
Sleep duration          (0, 0)  (0, 0)    (7.44, 0.67)    (7.39, 0.91)   
Sleep efficiency        (0, 0)  (0, 0)    (0.76, 0.14)    (0.81, 0.12)   
REM sleep percentage    (0, 0)  (0, 0)   (22.16, 4.62)   (22.36, 3.77)   
Deep sleep percentage   (0, 0)  (0, 0)  (51.43, 18.84)  (55.12, 13.94)   
Light sleep percentage  (0, 0)  (0, 0)  (27.62, 16.85)  (22.59, 13.27)   
Awakenings              (0, 0)  (0, 0)    (1.89, 1.42)    (1.63, 1.32)   
Caffeine consumption    (0, 0)  (0, 0)  (25.21, 43.43)  (13.97, 30.65)   
Alcohol consumption     (0, 0)  (0, 0)    (2.01, 1.93)    (1.02, 1.51)   
Smoking status          (0, 0)  (0, 0)    (0.38, 0.49)     (0.43, 0.5)   
Exercise frequency      (0, 0)  (0, 0)    (2.17, 1.03)    (2.12, 1.27)   

                              Gro